In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/christinedk/wp_internship/collaboration/')

In [6]:
from time  import time
from data_export import getTemplatesRegexRelaibility, getTemplatesRegex
from pyspark.sql.functions import udf, col, explode, regexp_replace

In [7]:
TEMPLATES = ['weasel','peacock','autobiography','advert','fanpov']
outputHDFS = 'page_history'
!hadoop fs -mkdir $outputHDFS

mkdir: `page_history': File exists


# Get all revisions with templates from WikiText

In [ ]:
snapshot ="2020-09"
wikidb = "enwiki"
wikitext_history = spark.sql('''SELECT page_id,page_title,revision_id,revision_text,user_id
    FROM wmf.mediawiki_wikitext_history 
    WHERE snapshot ="{snapshot}" and wiki_db ="{wikidb}"'''.format(wikidb=wikidb,snapshot=snapshot))

## Apply getTemplatesRegexRelaibility over all wikitext history
wikitext_history = wikitext_history.withColumn("templates",getTemplatesRegexRelaibility(col('revision_text')))

revisions_with_template = wikitext_history.select(wikitext_history.page_id,wikitext_history.page_title,wikitext_history.user_id,wikitext_history.revision_id,explode(wikitext_history.templates))
revisions_with_template = revisions_with_template.withColumn('page_title', regexp_replace('page_title', ' ', '_'))

## Persist outputs
revisions_with_template.write.parquet(outputHDFS+'/templates.parquet',mode='overwrite')

## Read back revisions with template
revisions_with_template = spark.read.parquet(outputHDFS+'/templates.parquet')

# Get full edit history of pages and editors

In [ ]:
## Select subset of mediawiki history containing all page titles with revisions

pages_templates_subset = revisions_with_template.select('page_title').distinct()
pages_templates_subset.createOrReplaceTempView('pages_templates_subset')

mediawiki_history_subset =  spark.sql('''
        SELECT w.event_timestamp, w.page_title,w.page_id,w.page_namespace, 
        w.revision_id, w.revision_is_identity_reverted, 
        w.revision_minor_edit, w.revision_text_bytes, 
        w.revision_first_identity_reverting_revision_id, w.revision_seconds_to_identity_revert,
        w.event_user_id,w.event_user_registration_timestamp, 
        w.event_user_is_anonymous,w.event_user_revision_count,

        w.event_comment
        FROM wmf.mediawiki_history w
        WHERE w.snapshot ="2020-09" and w.wiki_db ="enwiki" AND  
      w.event_entity = 'revision' AND w.page_title IN (
                    SELECT  page_title FROM pages_templates_subset)                   
        ''')
mediawiki_history_subset.cache()
mediawiki_history_subset.createOrReplaceTempView('mediawiki_history_subset')

In [9]:
## Get full histories of these pages *and* all editors who revised them

for template in templates:
    try:
        t1 = time()
        print(template)
        df = revisions_with_template.where(revisions_with_template['col']==template) # 
        df.cache()
        t2 = time()
        print('read table, done',t2-t1)
        t1 = time()
        page_ids = df.select('page_title').distinct()
        page_ids.createOrReplaceTempView('tmp_page_ids')
        revision_ids = df.select('revision_id').distinct()
        revision_ids.createOrReplaceTempView('tmp_revision_ids')
        reverts = spark.sql('''
        SELECT w.event_timestamp, w.page_title,w.page_id, w.page_namespace,
        w.revision_id, w.revision_is_identity_reverted, 
        w.revision_minor_edit, w.revision_text_bytes, 
        w.revision_first_identity_reverting_revision_id, w.revision_seconds_to_identity_revert,
        w.event_user_id,w.event_user_registration_timestamp, 
        w.event_user_is_anonymous,w.event_user_revision_count,
        CASE WHEN r.revision_id IS NOT NULL  THEN 1 ELSE 0 END has_template,
        w.event_comment
        FROM mediawiki_history_subset w 
        LEFT OUTER JOIN tmp_revision_ids r 
        ON (w.revision_id = r.revision_id)
        WHERE  w.page_title IN (SELECT page_title FROM tmp_page_ids) 
        ORDER BY page_title, w.revision_id
        ''') 
        reverts.repartition(1).write.format('json').save(outputHDFS+'/'+template,mode='overwrite')
        reverts.cache()
        t2 = time()
        print('save table, done',t2-t1)
        t1 = time()   
        templateout = template.replace(' ','_')
        !hadoop fs -text "$outputHDFS/$template/*" > $outputHDFS-$template-meta-info.json
        t2 = time()
        print('-----',t2-t1)
        
        print('extracting user histories')
        outputHDFS = 'user_history'
        reverts.createOrReplaceTempView('full_page_history')
        user_ids = full_page_history.select('event_user_id').distinct()
        user_ids.createOrReplaceTempView('tmp_user_ids')
        user_histories =  spark.sql('''
        SELECT w.event_timestamp, w.page_title,w.page_id,w.page_namespace, 
        w.revision_id, w.revision_is_identity_reverted, 
        w.revision_minor_edit, w.revision_text_bytes, 
        w.revision_first_identity_reverting_revision_id, w.revision_seconds_to_identity_revert,
        w.event_user_id,w.event_user_registration_timestamp, 
        w.event_user_is_anonymous,w.event_user_revision_count,
        w.event_comment
        FROM wmf.mediawiki_history w
        WHERE w.snapshot ="2020-09" and w.wiki_db ="enwiki" AND  
        w.event_entity = 'revision' 
        AND w.event_user_id IN (SELECT event_user_id FROM tmp_user_ids)                   
        ''')
        
        user_histories.repartition(1).write.format('json').save(outputHDFS+'/'+template,mode='overwrite')
        t2 = time()
        print('save table, done',t2-t1)
        t1 = time()   
        templateout = template.replace(' ','_')
        !hadoop fs -text "$outputHDFS/$template/*" > $outputHDFS-$template-meta-info.json
        t2 = time()
        print('-----',t2-t1)
        
        
    except Exception as e:
        print('error',e)

weasel
read table, done 0.07624244689941406
save table, done 261.32771944999695
21/02/23 16:41:27 INFO compress.CodecPool: Got brand-new decompressor [.snappy]
----- 19.260557174682617
peacock
read table, done 0.0747826099395752
save table, done 62.637932538986206
21/02/23 16:42:49 INFO compress.CodecPool: Got brand-new decompressor [.snappy]
----- 10.20436406135559
autobiography
read table, done 0.06866979598999023
save table, done 42.912111043930054
21/02/23 16:43:43 INFO compress.CodecPool: Got brand-new decompressor [.snappy]
----- 3.1491854190826416
advert
read table, done 0.06957197189331055
save table, done 84.30075573921204
21/02/23 16:45:10 INFO compress.CodecPool: Got brand-new decompressor [.snappy]
----- 27.113717317581177
fanpov
read table, done 0.07831907272338867
save table, done 35.95526933670044
21/02/23 16:46:13 INFO compress.CodecPool: Got brand-new decompressor [.snappy]
----- 3.868117570877075


In [8]:
ls -lh tmp-*

-rw-r--r-- 1 christinedk wikidev 725M Feb 23 11:05 tmp-advert-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 7.0G Feb 23 11:55 tmp-advert-meta-info.json
-rw-r--r-- 1 christinedk wikidev  39M Feb 23 11:00 tmp-autobiography-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 371M Feb 23 11:53 tmp-autobiography-meta-info.json
-rw-r--r-- 1 christinedk wikidev  64M Feb 23 11:08 tmp-fanpov-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 687M Feb 23 11:55 tmp-fanpov-meta-info.json
-rw-r--r-- 1 christinedk wikidev 349M Feb 23 10:59 tmp-peacock-meta-info.csv.gz
-rw-r--r-- 1 christinedk wikidev 3.5G Feb 23 11:53 tmp-peacock-meta-info.json
-rw-r--r-- 1 christinedk wikidev 731M Feb 23 10:56 tmp-weasel-meta-info.csv.gz
